In [2]:
# libraries
#!pip install rpy2
#!pip install pandas
#!pip install keras

    100% |████████████████████████████████| 184kB 518kB/s 
  Running setup.py bdist_wheel for rpy2 ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
  Stored in directory: /Users/Kozodoi/Library/Caches/pip/wheels/23/9e/ee/0e5f6a00aafef9935d40ebf7657278220139f0101321e30d07
Successfully built rpy2


In [1]:
# libraries
import rpy2.robjects as robjects
import pandas as pd
import keras
from keras.layers import Input, Dense, Embedding, concatenate, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
# load the data
path = "/Users/lizzzi111/Desktop/"
known   = pd.read_csv("Data/train.csv")
unknown = pd.read_csv("Data/test.csv")

In [3]:
# keep only Flow songs
known = known.query("listen_type == 1")
known.shape

(2319611, 15)

In [4]:
known['train'] = 1
unknown['train'] = 0
#full = [known, unknown]
#result = pd.concat(full)

### Data Preparation

In [5]:
# Prepare the data
# Create a placeholder for the IDs new in the test data
newUsers   = list(unknown.user_id[~unknown.user_id.isin(known.user_id)])       + list(known.user_id.value_counts().keys()[known.user_id.value_counts()     == 1])
newSongs   = list(unknown.media_id[~unknown.media_id.isin(known.media_id)])    + list(known.media_id.value_counts().keys()[known.media_id.value_counts()   == 1])[-1000:]
newArtists = list(unknown.artist_id[~unknown.artist_id.isin(known.artist_id)]) + list(known.artist_id.value_counts().keys()[known.artist_id.value_counts() == 1])[-1000:]
newGenres  = list(unknown.genre_id[~unknown.genre_id.isin(known.genre_id)])    + list(known.genre_id.value_counts().keys()[known.genre_id.value_counts()   == 1])[-1000:]

# In particular, assign IDs from 0 to N for users and songs
# Use enumerate() to create a list of the new and original IDs
users   = list(enumerate([i for i in known.user_id.unique()   if i not in newUsers]))
songs   = list(enumerate([i for i in known.media_id.unique()  if i not in newSongs]))
artists = list(enumerate([i for i in known.artist_id.unique() if i not in newArtists]))
genres  = list(enumerate([i for i in known.genre_id.unique()  if i not in newGenres]))

# Create a dictionary with old IDs to new IDs
userid2idx   = {o:i for i,o in users}
songid2idx   = {o:i for i,o in songs}
artistid2idx = {o:i for i,o in artists}
genreid2idx  = {o:i for i,o in genres}

# Update with new/rare entries
userid2idx.update({o:(max(userid2idx.values())+1)     for o in newUsers})
songid2idx.update({o:(max(songid2idx.values())+1)     for o in newSongs})
artistid2idx.update({o:(max(artistid2idx.values())+1) for o in newArtists})
genreid2idx.update({o:(max(genreid2idx.values())+1)   for o in newGenres})

# Create id variable with the new IDs
known['userIdx']   = known.user_id.apply(lambda x:       userid2idx[x])
known['songIdx']   = known.media_id.apply(lambda x:      songid2idx[x])
known['artistIdx'] = known.artist_id.apply(lambda x:     artistid2idx[x])
known['genreIdx']  = known.genre_id.apply(lambda x:      genreid2idx[x])
unknown['userIdx']   = unknown.user_id.apply(lambda x:   userid2idx[x])
unknown['songIdx']   = unknown.media_id.apply(lambda x:  songid2idx[x])
unknown['artistIdx'] = unknown.artist_id.apply(lambda x: artistid2idx[x])
unknown['genreIdx']  = unknown.genre_id.apply(lambda x:  genreid2idx[x])

In [6]:
# partition train/test data
ts = known.groupby(["userIdx"]).tail(5)
tr = known.groupby(["userIdx"], group_keys=False).apply(lambda x: x[:-5])

# move songs that appear only in ts to tr
strayObs = ts.songIdx.isin(tr.songIdx) & ts.userIdx.isin(tr.userIdx) & ts.artistIdx.isin(tr.artistIdx) & ts.genreIdx.isin(tr.genreIdx)
tr = tr.append(ts[~strayObs])
ts = ts[strayObs]

In [7]:
tr.platform_name.nunique()

3

In [22]:
# create an input layer with one row of IDs
user_in   = Input(shape = (1,), dtype='int64', name = "user_in")
song_in   = Input(shape = (1,), dtype='int64', name = "song_in")
artist_in = Input(shape = (1,), dtype='int64', name = "artist_in")
genre_in  = Input(shape = (1,), dtype='int64', name = "genre_in")
plat_in = Input(shape = (1,), dtype='int64', name = "plat_in")
# Create an embedding assigning k latent factors to each ID
# These will be optimized
# A regulariztaion is added to avoid very large weights
n_users   = tr.userIdx.nunique()
n_songs   = tr.songIdx.nunique()
n_artists = tr.artistIdx.nunique()
n_genres  = tr.genreIdx.nunique()
n_plat = tr.platform_name.nunique()

u = Embedding(n_users,   50, input_length=1, embeddings_regularizer=l2(1e-5))(user_in)
s = Embedding(n_songs,   50, input_length=1, embeddings_regularizer=l2(1e-5))(song_in)
a = Embedding(n_artists, 50, input_length=1, embeddings_regularizer=l2(1e-5))(artist_in)
g = Embedding(n_genres,  50, input_length=1, embeddings_regularizer=l2(1e-5))(genre_in)
#p = Embedding(n_plat,  50, input_length=1, embeddings_regularizer=l2(1e-5))(plat_in)
# Specify what to do with the layers
# We want to multiply them into a 'rating' matrix
x = concatenate([u, s, a, g])
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
#x = Dropout(0.5)(Dense(128, activation='relu')(x))
#x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x) 
x = Dropout(0.5)(x)
x = Dense(1, activation = "sigmoid")(x)
#x = merge([x, ub], mode = 'sum')
#x = merge([x, sb], mode = 'sum') # Can this be included in the line above?

# Then we specify the model that we want to use
model = Model([user_in, song_in, artist_in, genre_in, plat_in], x) # 
model.compile(Adam(0.001), loss="binary_crossentropy", metrics = ['accuracy'])

In [21]:
# run the estimations
model.fit([tr.userIdx, tr.songIdx, tr.artistIdx, tr.genreIdx, tr.platform_name], tr.is_listened,  #
validation_data = ([ts.userIdx, ts.songIdx, ts.artistIdx, ts.genreIdx, ts.platform_name], ts.is_listened),
batch_size = 22300, epochs = 10)

Train on 2259802 samples, validate on 59809 samples
Epoch 1/10
2259802/2259802 [==============================] - 85s - loss: 0.5968 - acc: 0.7013 - val_loss: 0.6802 - val_acc: 0.5730

KeyboardInterrupt: 

In [10]:
b

,sample_id,is_listened
0,0,0.977808
1,1,0.695002
2,2,0.635692
3,3,0.318223
4,4,0.975992


In [12]:
# saving submission
pred.to_csv(path + "deep_128_64_flow_liza_platform.csv", index = False)

In [23]:
model.fit([known.userIdx, known.songIdx, known.artistIdx, known.genreIdx, known.platform_name], known.is_listened,
batch_size = 22300, epochs = 10)

Epoch 1/10
2319611/2319611 [==============================] - 95s - loss: 0.5968 - acc: 0.7005      

In [24]:
# predict on unlabelled set
pred_wh = pd.DataFrame()
pred_wh["sample_id"] = unknown.sample_id
pred_wh["is_listened"] = model.predict([unknown.userIdx, unknown.songIdx, unknown.artistIdx, unknown.genreIdx, unknown.platform_name])
pred_wh.head(5)

,sample_id,is_listened
0,0,0.990724
1,1,0.782819
2,2,0.828149
3,3,0.494004
4,4,0.987979


In [16]:
# saving submission
pred_wh.to_csv(path + "deep_128_64_flow_liza_ap.csv", index = False)

In [25]:
# loading naive submission
naive = pd.read_csv(path + "naive_ratio_user.csv")

In [26]:
# combining predictions
pred_mean = pred_wh
pred_mean["is_listened"] = 0.5*pred_wh["is_listened"] + 0.5*naive["is_listened"]
pred_mean.head(5)

,sample_id,is_listened
0,0,0.995362
1,1,0.778415
2,2,0.853833
3,3,0.549430
4,4,0.939891


In [27]:
pred_mean.to_csv(path + "deep_128_64_flow_liza_ap_naive05_var.csv", index = False)